In [1]:
% cd /Users/kimia/Downloads/

/Users/kimia/Downloads


In [2]:
import pandas as pd



votes = pd.read_csv('BRO_H_20170110.txt', sep="\t", header = None)
reg = pd.read_csv('BRO_20170110.txt', sep="\t", header = None)


/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
reg["id"]= reg[1]
votes["id"]= votes[1]


In [4]:
## only general election for now
votes = votes.loc[votes[3]=="GEN"]

votes = votes.loc[votes[2]=="11/08/2016"]

In [5]:
#df1.merge(df2,how='left', left_on='Column1', right_on='ColumnA') 
registration = reg.merge(votes, how = "left", left_on='id', right_on='id', suffixes=['reg','votes'])
print(len(registration["id"].unique()), len(registration["id"]))
# make sure not more than 1 ID per file

registration.drop([5,6,7,8,9, 12, 13, 14, 15, 16, 17, 18, 20, 21, 24, 25, 26, 27,28,29,30,31,32,33, 34,35, 36, 37], axis=1, inplace=True)
newcolumns = ["district", "id", "last", "dr0p", "first", "garbage", "zip", "female", "dob", "party", "genid", "drop2", "idk", "electiondate", "general", "typeofvote"]
registration.columns = newcolumns 

1214895 1214895


In [6]:
#registration.iloc[0:5,21:32]

In [7]:
registration.head()

,district,id,last,dr0p,first,garbage,zip,female,dob,party,genid,drop2,idk,electiondate,general,typeofvote
0,BRO,101223429,Mowery,NaN,Ethel,,330647802,F,02/03/1967,REP,101223429,NaN,NaN,NaN,NaN,NaN
1,BRO,101220263,Dadson,NaN,James,,333080000,M,10/05/1968,REP,101220263,NaN,NaN,NaN,NaN,NaN
2,BRO,101231790,Barone,NaN,Dolores,,330620000,F,03/19/1970,REP,101231790,BRO,101231790.0,11/08/2016,GEN,Y
3,BRO,101235763,Bibey,NaN,Eleanor,,330620000,F,09/28/1971,REP,101235763,BRO,101235763.0,11/08/2016,GEN,Y
4,BRO,101237948,Turner,NaN,Charlene,,333082928,F,01/21/1972,REP,101237948,BRO,101237948.0,11/08/2016,GEN,Y


In [8]:

#clean general election to 0/1
registration["general"] = registration["general"].fillna(0)

registration.loc[registration["general"]=="GEN","general"] = 1

#registration = registration.fillna(0)

#clean gener
registration.loc[registration["female"]=="F","female"] = 1
registration.loc[registration["female"]=="M","female"] = 0
#typeofvote
registration["typeofvote"] = registration["typeofvote"].fillna(0)


In [9]:
import numpy as np
print((np.sum(registration["general"]))/ (len(registration)), "Turnout!")


0.6847118475259178 Turnout!


In [10]:
### testing out a model

In [11]:
df = registration
df.columns
remove = ['district', 'id', 'last', 'dr0p', 'first', 'garbage', 
       'dob', 'genid', 'drop2', 'idk', 'electiondate',
       'typeofvote']
df.drop(remove, axis = 1, inplace = True)


In [12]:
df = pd.get_dummies(df, columns=['party'])

In [17]:
msk = np.random.rand(len(df)) < 0.5
train = df[msk]
test = df[~msk]
y_train = train['general']
X_train = train[train.columns[train.columns != 'general']]
y_test = test['general']
X_test = test[test.columns[test.columns != 'general']]

In [18]:
from sklearn.linear_model import LassoCV
alpha=[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
lasso = LassoCV(alphas=alpha, cv=10, max_iter=10000, fit_intercept=True)
lasso.fit(X_train, y_train)
#y_hat_pca_train = lasso.predict(X_train)
#y_hat_pca_test = lasso.predict(X_test)
print(lasso.score(X_train, y_train), lasso.score(X_test,
y_test))

ValueError: could not convert string to float: 'U'